In [ ]:
import argparse
import random
import time
import sys
import re
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from markdownify import markdownify as md


In [ ]:
root_url = 'https://www.biomart.cn/'
max_retries = 5

# 重试间隔时间（秒）
retry_interval = 2

In [ ]:
def get_method(url3,title3,dir3):
    full_url = root_url + url3
    for attempt in range(max_retries):
        try:
            # 发送请求
            response4 = requests.get(full_url, timeout=5)  # 设置超时时间为 5 秒
            response4.raise_for_status()  # 如果响应状态码不是 200，抛出异常
            soup = BeautifulSoup(response4.text, 'html.parser')
            break  # 请求成功，退出循环
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                time.sleep(retry_interval)
            else:
                print("Max retries reached. Giving up.")
    #divs_with_ids  = soup.find_all('div', id=True)
    divs_with_contents  = soup.find_all('div', class_='rich-box')
    divs_with_ids = soup.find_all('div', class_='tw-flex tw-items-center tw-my-20')
    #print (len(divs_with_contents),len(divs_with_ids))
    result_id = []
    result_content = []
    for div in divs_with_ids:
        #div_id = div['id']
        #if div_id in ['__next','main-box','right-container','j-dxy-bottom']:continue
        paragraphs = div.find('p').get_text()
        paragraphs = re.sub(r'[^\w\u4e00-\u9fff\sa-zA-Z]', '', paragraphs).strip()
        result_id.append({
            'id': paragraphs
        })
    for div in divs_with_contents:
        div_html = str(div)
        markdown_content =  md(div_html)
        result_content.append({
            'markdown': markdown_content
        })

## 转markdown
    outfile_name = f"{dir3}/{title3}"
    if os.path.exists(f"{outfile_name}.md"):
        outfile_name = outfile_name + "_add"
    with open(f"{outfile_name}.md", "w", encoding="utf-8") as f:
        for i in range(0,len(result_content)):
            #f.write(f"## {item['id']}\n\n")
            f.write(f"## {result_id[i]['id']}\n\n")
            f.write(result_content[i]['markdown'])
            f.write("\n\n---\n\n")
    time.sleep(2)

In [ ]:
def exp_level2(url2,dir2):
    full_url = root_url + url2
    #print (full_url)
    for attempt in range(max_retries):
        try:
            # 发送请求
            response3 = requests.get(full_url, timeout=5)  # 设置超时时间为 5 秒
            response3.raise_for_status()  # 如果响应状态码不是 200，抛出异常
            soup = BeautifulSoup(response3.text, 'html.parser')
            break  # 请求成功，退出循环
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                time.sleep(retry_interval)
            else:
                print("Max retries reached. Giving up.")
    soup = BeautifulSoup(response3.text, 'html.parser')
    ## 找到Methods
    links  = soup.find_all('a', class_='tw-mb-10 tw-inline-block tw-w-full tw-cursor-pointer tw-rounded-8 tw-bg-other-400 tw-p-20 last:tw-mb-none hover:tw-bg-other-300 hover:tw-text-current md:tw-mb-20')
    for link in links:
        method_title = link.find('b').get_text()
        method_title = re.sub(r'[^\w\u4e00-\u9fff\sa-zA-Z]', '', method_title).strip()
        href = link.get('href')
        #text = link.get_text()
        if href:
            print(f'{href}' + '\t' + f'{method_title}')
            get_method(href,method_title,dir2)
        else:
            #pass
            print ('not found')
        time.sleep(1)
        #break
    

In [ ]:
empty_dir = pd.read_csv('/public/home/liujunwu/workdir/MEPC/ExperimentMethod/empty.dir',header=0)
empty_dir.columns = ['dir']
log_raw = pd.read_csv('/public/home/liujunwu/workdir/MEPC/ExperimentMethod/log_all.bak',header=None,sep='\t',on_bad_lines='skip')
log_raw.columns = ['url','base_dir']



In [ ]:
log_raw.head()

In [ ]:
empty_dir['base_dir'] = empty_dir['dir'].apply(lambda x: x.split('/')[-1])
print (empty_dir.shape)
empty_dir.head()

In [ ]:
merged_dir = pd.merge(empty_dir,log_raw,how='left',left_on='base_dir',right_on='base_dir')
print (merged_dir.shape)
merged_dir.head()

In [ ]:
merged_dir[merged_dir['url'].str.match('method', na=False)]

In [ ]:
merged_dir = merged_dir[merged_dir['url'].notnull()]
merged_dir.shape

In [ ]:
for index, row in merged_dir.iterrows():
    print (row['url'],row['dir'])
    exp_level2(row['url'],row['dir'])
    time.sleep(1)

In [ ]:
exp_level2('/lab-web/exp/316nrkggo40di/31o6i0ego4hki.html','/public/home/liujunwu/workdir/MEPC/ExperimentMethod/植物学实验/植物染色体核型分析')